# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san patricio,28.0170,-97.5169,23.97,74,54,6.07,US,1738801052
1,1,kavaratti,10.5669,72.6420,26.94,73,13,4.12,IN,1738801055
2,2,utrik,11.2278,169.8474,26.34,82,87,8.73,MH,1738801056
3,3,rio grande,-32.0350,-52.0986,25.25,92,20,8.20,BR,1738801057
4,4,kuanda,56.3161,116.0806,-37.49,89,31,1.36,RU,1738801058


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    alpha = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 21) &
    (city_data_df["Max Temp"] <= 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] <= 10), :]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,mount gambier,-37.8333,140.7667,23.71,49,7,1.86,AU,1738801114
62,62,ormond-by-the-sea,29.3491,-81.0665,21.05,91,0,2.57,US,1738801123
69,69,cabo san lucas,22.8909,-109.9124,25.17,31,0,3.09,MX,1738800945
96,96,cidreira,-30.1811,-50.2056,25.78,92,0,3.98,BR,1738801161
101,101,coahuayana de hidalgo,18.7000,-103.6583,26.98,67,0,2.19,MX,1738801167
103,103,las khorey,11.1595,48.1967,21.58,74,3,3.88,SO,1738801169
114,114,ambovombe,-25.1667,46.0833,24.02,90,9,1.53,MG,1738801181
177,177,tamiahua,21.2667,-97.4500,22.91,86,4,2.77,MX,1738801258
216,216,hadibu,12.6500,54.0333,21.86,72,7,4.11,YE,1738801305
330,330,crane,31.3974,-102.3501,25.49,21,0,1.42,US,1738801437


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\arsen\AppData\Local\Temp\ipykernel_35472\4207309733.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
53,mount gambier,AU,-37.8333,140.7667,49,
62,ormond-by-the-sea,US,29.3491,-81.0665,91,
69,cabo san lucas,MX,22.8909,-109.9124,31,
96,cidreira,BR,-30.1811,-50.2056,92,
101,coahuayana de hidalgo,MX,18.7000,-103.6583,67,
103,las khorey,SO,11.1595,48.1967,74,
114,ambovombe,MG,-25.1667,46.0833,90,
177,tamiahua,MX,21.2667,-97.4500,86,
216,hadibu,YE,12.6500,54.0333,72,
330,crane,US,31.3974,-102.3501,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":1,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mount gambier - nearest hotel: The Old Mount Gambier Gaol
ormond-by-the-sea - nearest hotel: Howard Johnson
cabo san lucas - nearest hotel: Comfort Rooms
cidreira - nearest hotel: Hotel Castelo
coahuayana de hidalgo - nearest hotel: No hotel found
las khorey - nearest hotel: No hotel found
ambovombe - nearest hotel: No hotel found
tamiahua - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
cerro de ortega - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
san julian - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
tokar - nearest hotel: No hotel found
kasaragod - nearest hotel: PWD Canteen Kasaragod
lerdo de tejada - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
53,mount gambier,AU,-37.8333,140.7667,49,The Old Mount Gambier Gaol
62,ormond-by-the-sea,US,29.3491,-81.0665,91,Howard Johnson
69,cabo san lucas,MX,22.8909,-109.9124,31,Comfort Rooms
96,cidreira,BR,-30.1811,-50.2056,92,Hotel Castelo
101,coahuayana de hidalgo,MX,18.7000,-103.6583,67,No hotel found
103,las khorey,SO,11.1595,48.1967,74,No hotel found
114,ambovombe,MG,-25.1667,46.0833,90,No hotel found
177,tamiahua,MX,21.2667,-97.4500,86,No hotel found
216,hadibu,YE,12.6500,54.0333,72,No hotel found
330,crane,US,31.3974,-102.3501,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display
# I've decided not to loc hotel_df to drop rows where hotel wasn't found since in the starter file it also hasn't been done
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)